In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# os.getcwd(), os.listdir()

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import normalize

In [ ]:
epsilon = 1e-6

In [ ]:
print(ResNet50().summary())

102967424/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                    

In [ ]:
# Load model once
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
desired_layer_name = 'avg_pool'
desired_layer_output = resnet_model.get_layer(desired_layer_name).output
feature_extraction_model = Model(inputs=resnet_model.input, outputs=desired_layer_output)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
import cv2
def process_and_extract_features(image_path):
    # Load the image
    loaded_image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if loaded_image is not None:
        # Resize the image to a common size
        new_size = (200, 300)
        resized_image = cv2.resize(loaded_image, new_size)

        # Adjust contrast
        alpha = 1.5
        adjusted_image = cv2.convertScaleAbs(resized_image, alpha=alpha, beta=0)

        # Apply geometrical orientation
        angle = 30
        rows, cols, _ = adjusted_image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_image = cv2.warpAffine(adjusted_image, rotation_matrix, (cols, rows))

        # Apply random flips
        flip_code = np.random.randint(-1, 2)
        flipped_image = cv2.flip(rotated_image, flip_code)

        # Adjust brightness
        brightness_factor = 1.2
        brightened_image = cv2.convertScaleAbs(flipped_image, alpha=brightness_factor, beta=0)

        # Modify exposure (Gamma Correction)
        gamma = 1.5
        exposure_adjusted_image = np.power(brightened_image / 255.0, gamma) * 255.0
        exposure_adjusted_image = np.uint8(exposure_adjusted_image)

In [ ]:
# Function to fetch image from URL
def fetch_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Function for image preprocessing
def preprocess_image(img):
    img = img.convert('RGB')  # Convert to RGB mode
    img = img.resize((224, 224))  # Resize to 224x224
    img = np.array(img)  # Convert to numpy array
    img = preprocess_input(img)  # Preprocess input according to ResNet50 requirements
    return img

# Function to extract image features using ResNet50
def extract_image_features(img):
    img = preprocess_input(img)  # Preprocess input according to ResNet50 requirements
    img_features = feature_extraction_model.predict(img.reshape(1, 224, 224, 3))
    return img_features.flatten()

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/A2_Data.csv')
data.fillna("",inplace=True)

# Define path to save the image features pickle file
image_features_path = '/content/drive/MyDrive/image_features.pkl'

# Extract image URLs from the 'Image' column
image_urls = data['Image']

# Load or Extract Image Features
if os.path.exists(image_features_path):
    with open(image_features_path, 'rb') as f:
        image_features = pickle.load(f)
else:
    # Image feature extraction
    image_features = []
    for url_list in data['Image']:

        urls = eval(url_list)
        url = urls[0]
        # url = url_list.strip("[]").replace("'", "").split(", ")[0]  # Extracting the URL string from the list
        try:
            img = fetch_image(url)
            img = preprocess_image(img)
            img_features = extract_image_features(img)
            image_features.append(img_features)
        except Exception as e:
            print(f"Error processing image from URL: {url}. Error: {e}")

    image_features = np.array(image_features)
    image_features = normalize(image_features)  # Normalize image features

    # Save image features using pickle
    with open(image_features_path, 'wb') as f:
        pickle.dump(image_features, f)

1/1 [==============================] - 0s 175ms/step
Error processing image from URL: https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg. Error: cannot identify image file <_io.BytesIO object at 0x7bf1e973d8f0>
1/1 [==============================] - 0s 292ms/step
Error processing image from URL: https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg. Error: cannot identify image file <_io.BytesIO object at 0x7bf1eb62de40>
1/1 [==============================] - 0s 305ms/step
Error processing image from URL: https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg. Error: cannot identify image file <_io.BytesIO object at 0x7bf1ebb515d0>
1/1 [==============================] - 0s 189ms/step
Error processing image from URL: https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg. Error: cannot identify image file <_io.BytesIO object at 0x7bf1eba75490>
1/1 [==============================] - 0s 174ms/step
Error processing image 

In [ ]:
from scipy.spatial.distance import cosine
from scipy import spatial

# Load image features
with open(image_features_path, 'rb') as f:
    all_image_features = pickle.load(f)

# Function to compute cosine similarity between two vectors
def compute_cosine_similarity(vec1, vec2):
    v1, v2 = np.array(vec1) + epsilon, np.array(vec2) + epsilon
    return 1 - spatial.distance.cosine(v1, v2)

# Function to find top similar items
def find_top_similar_items(input_feature, all_features, top_n=3):
    similarity_scores = [compute_cosine_similarity(input_feature, feature) for feature in all_features]
    top_similar_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    return top_similar_indices

# Function to print top similar items
def print_top_similar_items(top_similar_indices, feature_type):
    print(f"Using {feature_type} Retrieval:")
    for i, idx in enumerate(top_similar_indices, 1):
        image_url = image_urls[idx]
        similarity = similarity_scores[idx]
        print(f"{i}) Image URL: {image_url}")
        print(f"   Cosine Similarity: {similarity:.4f}")
        print()


# # Take user input for image URL
input_image_url = input("Enter the image URL: ")
# input_image_url = 'https://images-na.ssl-images-amazon.com/images/I/717mWpCp65L._SY88.jpg'

# Fetch image from URL and preprocess it
input_image = fetch_image(input_image_url)
input_image_feature = extract_image_features(preprocess_image(input_image))

# print("Input Image Features:")
# print(input_image_feature.shape)

# print("All Features:")
# print(all_image_features.shape)
# print(all_image_features)

# Compute cosine similarity between input image and all other images
similarity_scores = [compute_cosine_similarity(input_image_feature, feature) for feature in all_image_features]
top_similar_images = find_top_similar_items(input_image_feature, all_image_features)
print_top_similar_items(top_similar_images, "Image")

Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
1/1 [==============================] - 0s 177ms/step
Using Image Retrieval:
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
   Cosine Similarity: 1.0000

2) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71ssCRcurhL._SY88.jpg']
   Cosine Similarity: 0.7639

3) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/719n906u2fL._SY88.jpg']
   Cosine Similarity: 0.7572



In [ ]:
import nltk
import numpy as np
import pandas as pd
import string
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter

# First, ensure you've downloaded necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/A2_Data.csv')

# Function for text preprocessing
def preprocess_text(text):
    # Check if text is a string
    if isinstance(text, str):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in string.punctuation]
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        return tokens
    else:
        return []  # Return empty list if text is not a string

# Process textual reviews
data['Processed_Review'] = data['Review Text'].apply(preprocess_text)

# Instead of CSV, use JSON to save the processed data, preserving list structure
data.to_json('/content/drive/MyDrive/Preprocessed_Data.json', orient='records', lines=True)

# Load preprocessed data
data = pd.read_json('/content/drive/MyDrive/Preprocessed_Data.json', lines=True)

# Function to calculate TF (Term Frequency)
def calculate_tf(tokens):
    tf_counter = Counter(tokens)
    total_words = len(tokens)
    tf = {word: tf_counter[word] / total_words for word in tf_counter}
    return tf

# Function to calculate IDF (Inverse Document Frequency)
def calculate_idf(documents):
    idf_counter = Counter()
    for document in documents:
        idf_counter.update(set(document))
    num_documents = len(documents)
    idf = {word: np.log(num_documents / (idf_counter[word] + 1)) for word in idf_counter}
    return idf

# Function to calculate TF-IDF scores
def calculate_tfidf(tf, idf):
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf

# Prepare data for TF-IDF calculations
all_processed_reviews = data['Processed_Review'].tolist()

# Calculate IDF using all processed reviews
idf = calculate_idf(all_processed_reviews)

# For each review, calculate TF and then TF-IDF
for index, review in enumerate(all_processed_reviews):
    tf = calculate_tf(review)
    tfidf = calculate_tfidf(tf, idf)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import ast  # For safely evaluating strings containing lists

# Calculate IDF using all processed reviews
idf = calculate_idf(all_processed_reviews)

# Create an empty column for storing TF-IDF results in a more structured form
data['TF-IDF'] = np.nan

# For each review, calculate TF and then TF-IDF
for index, review in enumerate(all_processed_reviews):
    tf = calculate_tf(review)
    tfidf = calculate_tfidf(tf, idf)
# Create an empty dictionary to store TF-IDF scores for all reviews
all_tfidf_dict = {}

# For each review, calculate TF and then TF-IDF
for index, review in enumerate(all_processed_reviews):
    tf = calculate_tf(review)
    tfidf = calculate_tfidf(tf, idf)

    # Store TF-IDF scores for the current review in the dictionary
    all_tfidf_dict[f"Review {index+1}"] = tfidf

# Print TF-IDF scores for all reviews in horizontal dictionary format
print("TF-IDF scores for all reviews:")
for review, tfidf in all_tfidf_dict.items():
    print(review, ":", tfidf)



TF-IDF scores for all reviews:
Review 1 : {'love': 0.13757121283490303, 'vintag': 0.5284421733087595, 'spring': 0.6437751649736402, 'strat': 0.20673951928078782, 'good': 0.10500243238111787, 'tension': 0.31403538010972787, 'great': 0.07575427705680808, 'stabil': 0.31403538010972787, 'float': 0.36809739452414975, 'bridg': 0.2234938144995149, 'want': 0.14303875627895873, 'way': 0.17357934571853118, 'go': 0.13916491422514002}
Review 2 : {'work': 0.060875746138159464, 'great': 0.04734642316050505, 'guitar': 0.052449210034205454, 'bench': 0.24204762459641782, 'mat': 0.24204762459641782, 'rug': 0.19188209108283716, 'enough': 0.10523869351284398, 'abus': 0.23006087157759358, 'take': 0.22907268296853878, 'care': 0.2868349480152009, 'make': 0.08468991482420773, 'organ': 0.21316649207308674, 'workspac': 0.2589420041009246, 'much': 0.09011763127578695, 'easier': 0.16096803505244642, 'screw': 0.11021147508086757, 'wo': 0.1408914480985823, "n't": 0.045196224312463264, 'roll': 0.1962721125685799, 'a

In [ ]:
# Load dataset
data = pd.read_csv('/content/drive/MyDrive/A2_Data.csv')

# Function for text preprocessing
def preprocess_text(text):
    # Check if text is a string
    if isinstance(text, str):
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in string.punctuation]
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        return tokens
    else:
        return []  # Return empty list if text is not a string

# Process textual reviews
data['Processed_Review'] = data['Review Text'].apply(preprocess_text)

# Instead of CSV, use JSON to save the processed data, preserving list structure
data.to_json('/content/drive/MyDrive/Preprocessed_Data.json', orient='records', lines=True)

# Load preprocessed data
data = pd.read_json('/content/drive/MyDrive/Preprocessed_Data.json', lines=True)

# Function to calculate TF (Term Frequency)
def calculate_tf(tokens):
    tf_counter = Counter(tokens)
    total_words = len(tokens)
    tf = {word: tf_counter[word] / total_words for word in tf_counter}
    return tf

# Function to calculate IDF (Inverse Document Frequency)
def calculate_idf(documents):
    idf_counter = Counter()
    for document in documents:
        idf_counter.update(set(document))
    num_documents = len(documents)
    idf = {word: np.log(num_documents / (idf_counter[word] + 1)) for word in idf_counter}
    return idf

# Function to calculate TF-IDF scores
def calculate_tfidf(tf, idf):
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf

# Prepare data for TF-IDF calculations
all_processed_reviews = data['Processed_Review'].tolist()

# Calculate IDF using all processed reviews
idf = calculate_idf(all_processed_reviews)

# Create an empty column for storing TF-IDF results in a more structured form
data['TF-IDF'] = np.nan

# For each review, calculate TF and then TF-IDF
for index, review in enumerate(all_processed_reviews):
    tf = calculate_tf(review)
    tfidf = calculate_tfidf(tf, idf)

    # Store TF-IDF scores for the current review in the dataframe
    data.at[index, 'TF-IDF'] = json.dumps(tfidf)  # Storing the results as a JSON string for compatibility

# Define path to save the image features pickle file
image_features_path = '/content/drive/MyDrive/image_features.pkl'

# Extract image URLs from the 'Image' column
image_urls = data['Image']

# Load or Extract Image Features
if os.path.exists(image_features_path):
    with open(image_features_path, 'rb') as f:
        image_features = pickle.load(f)
else:
    # Load model once
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    desired_layer_name = 'avg_pool'
    desired_layer_output = resnet_model.get_layer(desired_layer_name).output
    feature_extraction_model = Model(inputs=resnet_model.input, outputs=desired_layer_output)

    # Image feature extraction
    image_features = []
    for url_list in data['Image']:
        urls = eval(url_list)
        url = urls[0]
        try:
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            img = img.convert('RGB')  # Convert to RGB mode
            img = img.resize((224, 224))  # Resize to 224x224
            img = np.array(img)  # Convert to numpy array
            img = preprocess_input(img)  # Preprocess input according to ResNet50 requirements
            img_features = feature_extraction_model.predict(img.reshape(1, 224, 224, 3))
            img_features = img_features.flatten()
            image_features.append(img_features)
        except Exception as e:
            print(f"Error processing image from URL: {url}. Error: {e}")

    image_features = np.array(image_features)
    image_features = normalize(image_features)  # Normalize image features

    # Save image features using pickle
    with open(image_features_path, 'wb') as f:
        pickle.dump(image_features, f)

# Function to fetch image from URL
def fetch_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Function for image preprocessing
def preprocess_image(img):
    img = img.convert('RGB')  # Convert to RGB mode
    img = img.resize((224, 224))  # Resize to 224x224
    img = np.array(img)  # Convert to numpy array
    img = preprocess_input(img)  # Preprocess input according to ResNet50 requirements
    return img

# Function to extract image features using ResNet50
def extract_image_features(img):
    img = preprocess_input(img)  # Preprocess input according to ResNet50 requirements
    img_features = feature_extraction_model.predict(img.reshape(1, 224, 224, 3))
    return img_features.flatten()

# Load image features
with open(image_features_path, 'rb') as f:
    all_image_features = pickle.load(f)

# Function to compute cosine similarity between two vectors
def compute_cosine_similarity(vec1, vec2):
    v1, v2 = np.array(vec1) + epsilon, np.array(vec2) + epsilon
    return 1 - spatial.distance.cosine(v1, v2)

# Function to find top similar items
def find_top_similar_items(input_feature, all_features, top_n=3):
    similarity_scores = [compute_cosine_similarity(input_feature, feature) for feature in all_features]
    top_similar_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    return top_similar_indices, similarity_scores

def vectorize(tf_idf, all_words):
    sz = len(all_words)
    v = np.zeros((sz,))
    for word, val in tf_idf.items():
        if word in all_words:
            idx = all_words.index(word)
            v[idx] = val
    return v

# Function to print top similar items
def print_top_similar_items(top_similar_indices, image_similarity_scores, text_similarity_scores, feature_type):
    print(f"USING {feature_type} RETRIEVAL:")
    print("Top Similar Items:")
    # for i, idx in enumerate(top_similar_indices, 1):
    for i in range(3):
        idx = top_similar_indices[i]
        print(f"Idx:{idx} ImageSimilarity:{image_similarity_scores[idx]} TextSimilarity:{text_similarity_scores[idx]}")
        print(f"  URL:", data.iloc[idx, 1])
        print(f"  Review:", data.iloc[idx, 2])
        c = (image_similarity_scores[idx] + text_similarity_scores[idx]) / 2
        print(f"  Composite: {c}")

# Take user input for image URL and review text
#input_image_url = 'https://images-na.ssl-images-amazon.com/images/I/71Isri9SEaL._SY88.jpg'
#input_review_text = "Great price and good quality.  It didn't quite match the radius of my sound hole but it was close enough."
input_image_url = input("Enter the image URL: ")
input_review_text = input("Enter the review text: ")

# Fetch image from URL and preprocess it
input_image = fetch_image(input_image_url)
input_image_feature = extract_image_features(preprocess_image(input_image))

# Process input review text
processed_input_review = preprocess_text(input_review_text)

# Update
all_words = sorted(list(idf.keys()))
# Calculate TF-IDF for input review text
input_tfidf = calculate_tfidf(calculate_tf(processed_input_review), idf)
vectorized_input = vectorize(input_tfidf, all_words)

# Compute cosine similarity between input image and all other images
top_similar_images, image_similarity_scores = find_top_similar_items(input_image_feature.flatten(), all_image_features, top_n=4)

# Update to compute similarity scores
text_similarity_scores = []
for review_tfidf_json in data['TF-IDF']:
    review_tfidf = json.loads(review_tfidf_json)  # Make sure to convert the JSON string back into a dictionary.
    review_vector = vectorize(review_tfidf, all_words)  # Use the defined 'vectorize' function.
    score = compute_cosine_similarity(vectorized_input, review_vector)  # Correct variable name here.
    text_similarity_scores.append(score)

top_similar_texts_indices = np.argsort(text_similarity_scores)[-3:][::-1]

print_top_similar_items(top_similar_images, image_similarity_scores, text_similarity_scores, "Image")
print()
print_top_similar_items(top_similar_texts_indices, image_similarity_scores, text_similarity_scores, "Text")

Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/716PE4iRuFL._SY88.jpg
Enter the review text: I have never used AKG microphones before. I decided to take a chance on this from all the great reviews. They were right! I used this to record a Taylor 914CE, the sound was chrisp, great mid and just enough low to not overpower the sound. I totally recommend this for vocals and acoustic guitar recording!
1/1 [==============================] - 0s 182ms/step
USING Image RETRIEVAL:
Top Similar Items:
Idx:20 ImageSimilarity:1 TextSimilarity:1
  URL: ['https://images-na.ssl-images-amazon.com/images/I/716PE4iRuFL._SY88.jpg']
  Review: I have never used AKG microphones before. I decided to take a chance on this from all the great reviews. They were right! I used this to record a Taylor 914CE, the sound was chrisp, great mid and just enough low to not overpower the sound. I totally recommend this for vocals and acoustic guitar recording!
  Composite: 1.0
Idx:403 ImageSimilarity:0

In [ ]:

# Sample data structure for storing composite scores along with URLs and review texts
composite_list = []

# Assuming `data` is DataFrame containing URLs, reviews, etc.
for index, url_list in enumerate(image_urls):
    url = eval(url_list)[0] if isinstance(url_list, str) else url_list
    review_text = data.iloc[index]['Review Text']

    # Check if index is within bounds of image_similarity_scores and text_similarity_scores
    if index < len(image_similarity_scores) and index < len(text_similarity_scores):
        image_sim_score = image_similarity_scores[index]
        text_sim_score = text_similarity_scores[index]
        composite_similarity = (image_sim_score + text_sim_score) / 2  # Averaging the scores

        composite_list.append({
            "url": url,
            "text": review_text,
            "cosine_image": image_sim_score,
            "cosine_text": text_sim_score,
            "composite_similarity": composite_similarity
        })

# Storing composite list using pickle
import pickle
with open('/content/drive/MyDrive/composit_list.pkl', 'wb') as f:
    pickle.dump(composite_list, f)

# Loading and sorting the composite list
with open('/content/drive/MyDrive/composit_list.pkl', 'rb') as f:
    composite_list = pickle.load(f)

# Sorting composite list based on the composite similarity score
sorted_list_composite = sorted(composite_list, key=lambda x: x['composite_similarity'], reverse=True)

# Getting and printing the top 3 entries
top_three_composite = sorted_list_composite[:3]
for index, item in enumerate(top_three_composite, 1):
    print(f"({index})")
    print(f"Image URL: [\'{item['url']}\']")
    print(f"Review: {item['text']}")
    print(f"Cosine similarity of text: {item['cosine_text']:.4f}")
    print(f"Cosine similarity of images: {item['cosine_image']:.4f}")
    print(f"Composite similarity score: {item['composite_similarity']:.4f}")
    print()


(1)
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71Md5ihUFLL._SY88.jpg']
Review: We use these for everything from our acoustic bass down to our ukuleles. I know there is a smaller model available for ukes, violins, etc.; we haven't yet ordered those, but these will work on smaller instruments if one doesn't extend the feet to their maximum width. They're gentle on the instruments, and the grippy material keeps them secure.

The greatest benefit has been when writing music at the computer and needing to set a guitar down to use the keyboard/mouse - just easier for me than a hanging stand.

We have several and gave one to a friend for Christmas as well. I've used mine on stage, and it folds up small enough to fit right in my gig bag.
Cosine similarity of text: 1.0000
Cosine similarity of images: 1.0000
Composite similarity score: 1.0000

(2)
Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71aPXafkSaL._SY88.jpg']
Review: None
Cosine similarity of text: 0.098